In [2]:
import pandas as pd
import numpy as np
import os 
import random
import tweepy
from os.path import exists
from time import strftime, localtime, sleep

In [3]:
#get authorization squared away first

# authorization of API key and API secret
auth = tweepy.OAuthHandler('Your API key here', 
                           'Your API secret here')
  
# set access to user's access key and access secret 
auth.set_access_token('Your access key here', 
                      'Your access secret here')

# calling the api with our authentication
api = tweepy.API(auth)

In [4]:
#this function does all of the actual grunt work & tweeting

def auto_tweet_manager(original_df_path, media_path,):
    
    remaining_nft_path = 'Remaining NFTs.p'
    tweet_log_path = 'Tweet Log.p'
    
    #if no path is provided for the remaining NFT df, assing one and create the df, otherwise load it
    if exists(remaining_nft_path):
        remaining_nfts = pd.read_pickle(remaining_nft_path)
    else:
        remaining_nfts = pd.read_pickle(original_df_path)
        
    #if no tweetlog was provided, create one, otherwise load it
    if exists(tweet_log_path):
        tweet_log = pd.read_pickle(tweet_log_path)
    else:
        tweet_log = pd.DataFrame(columns=['Tweet_Time','Tweet_Data','name'])        
        
    
    #list all of the NFT filenames available to tweet, from the remaining NFT dataframe
    #randomly list all of the NFT filenames available to tweet from the remaining NFT dataframe
    remaining_names = list(remaining_nfts['name'])
    random.shuffle(remaining_names)

    #choose one NFT name at random
    current_nft = random.choice(remaining_names)
    #double check this NFT is not already in the 'tweet_log'
    tweets_logged_names = tweet_log['name'].tolist()
    if current_nft in tweets_logged_names:
        raise Exception('This NFT was already tweeted!')
        
    #pull the NFT's info from the dataframe
    current_nft_data = remaining_nfts[remaining_nfts['name']==current_nft].values.tolist()[0]
    #now we have a list of name, rectangles, circles, lines, spunk, kick, & BG color, and it's index
    current_name = current_nft_data[0]
    current_rect = current_nft_data[1]
    current_circ = current_nft_data[2]
    current_line = current_nft_data[3]
    current_spunk = current_nft_data[4]
    current_kick = current_nft_data[5]
    current_bg = current_nft_data[6].capitalize()
    current_index = remaining_nfts[remaining_nfts['name']==current_nft].index.tolist()[0]

    #craft a tweet using this information

    #use plural descriptors if needed
    if current_rect == '1':
        plural_rect = 'rectangle'
    else:
        plural_rect = 'rectangles'
    if current_circ == '1':
        plural_circ = 'circle'
    else:
        plural_circ = 'circles'
    if current_line == '1':
        plural_line = 'line'
    else:
        plural_line = 'lines'

    #format our tweet using the NFT specific information
    text = f'"{current_name.split(".")[0]}" contains {current_rect} {plural_rect}, {current_circ} \
{plural_circ}, {current_line} {plural_line}, and has a {current_bg} background. It boasts a \
spunk rating of {current_spunk} & a kick rating of {current_kick}. Modern Abstractions #{current_index}. \
#NFTs #nftart #ModernAbstractions'

    #also determine the exact path to our media file
    current_path = f'{media_path}/{current_nft}'
    
    #create & publish the tweet
    tweet_data = api.update_status_with_media(text, current_path)

    #get the NFT data as a pandas series from the 'remaining_nfts' dataframe
    current_data = remaining_nfts[remaining_nfts['name']==current_nft]

    #add the NFT data to our 'tweet_log' dataframe
    tweet_log = tweet_log.append(current_data)

    #also add the time & date it was tweeted, and the tweet data itself
    tweet_log.at[current_index,'Tweet_Time'] = strftime("%B %-d, %Y @ %-I:%M %p %Z", localtime())
    tweet_log.at[current_index,'Tweet_Data'] = tweet_data

    #remove any duplicate entries
    tweet_log.drop_duplicates('name', inplace=True)

    #now remove this NFT entry from the 'remaining_NFTs' dataframe
    remaining_nfts.drop(current_index, inplace=True)
    
    #finally save hardcopy files for the 'tweet_log' & 'remaining_nft' dataframes
    remaining_nfts.to_pickle(remaining_nft_path)
    tweet_log.to_pickle(tweet_log_path)
    
    #also save them as CSV files
    remaining_nfts_csv_path = remaining_nft_path.split('.')[0]+'.csv'
    remaining_nfts.to_csv(remaining_nfts_csv_path)
    tweet_log_csv_path = tweet_log_path.split('.')[0]+'.csv'
    tweet_log.to_csv(tweet_log_csv_path)
    
    tweet_time = tweet_log.at[current_index,'Tweet_Time']
    end_message = f'Successfully tweeted about {current_name.split(".")[0]} on {tweet_time}'
    
    return end_message

In [ ]:
#now put the function to use, tweeting every ~15 minutes

original_df_path = 'Full DataFrame.p'
media_path = 'Images'

while True:
    auto_tweet_manager(original_df_path, media_path)
    time.sleep(1000)